In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_PROVINSI.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
parameter_ranges = [(0.01, 1), (0.001, 1)]

# Register genetic operators
toolbox.register("attr_float", random.uniform)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(*parameter_ranges[0]),
                  lambda: random.uniform(*parameter_ranges[1])), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 50
population = toolbox.population(n=population_size)

# Genetic algorithm parameters
crossover_rate = 1
mutation_rate = 0
num_generations = 100

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu: [0.3929193146180988, 0.11088179784553348], MAPE: 29.226496944384422
Individu: [0.8500851087077474, 0.1384551293192028], MAPE: 27.52878670507521
Individu: [0.10609860780084152, 0.1031279742518646], MAPE: 32.05458868113357
Individu: [0.09342492592416317, 0.6176025110691397], MAPE: 20.390900629871226
Individu: [0.5688183047244458, 0.476608804462928], MAPE: 20.390900629871226
Individu: [0.6625728173998713, 0.49004354109834924], MAPE: 20.390900629871226
Individu: [0.8844048995983044, 0.07559925002111106], MAPE: 29.248998787656628
Individu: [0.5357597174531774, 0.48353125994494195], MAPE: 20.390900629871226
Individu: [0.03159323364658449, 0.13769590478829388], MAPE: 34.8033322726715
Individu: [0.5587588892617364, 0.8118874718406534], MAPE: 20.390900629871226
Individu: [0.12542951666736907, 0.5289991968463075], MAPE: 20.390900629871226
Individu: [0.011335280256051558, 0.8255707793395215], MAPE: 20.390900629871226
Individu: [0.46857345869585115, 0.5092706142590224], MAPE:

In [6]:
# Nilai minimum dan maksimum asli dari data yang digunakan untuk normalisasi
data_min = 3  # Sesuaikan dengan nilai minimum data asli Anda
data_max = 36  # Sesuaikan dengan nilai maksimum data asli Anda

# Denormalisasi secara manual
y_pred_denormalized = y_pred * (data_max - data_min) + data_min

#pred_denormalized =  MinMaxScaler.inverse_transform(y_pred.reshape(1, -1))
print(y_pred_denormalized)

[18.40928946 18.14982008 18.24529591 18.17992069 18.05034959 18.24529591
 18.40928946 18.512467   18.14982008 18.01302051 18.52196519 18.17992069]
